In [2]:
import torch
from models.common import AutoShape, DetectMultiBackend
from pathlib import Path
import json

/home/dannya1/anaconda3/envs/torch2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_next(file):
    score, density, config = 0, 0, {}
    is_conf = False
    for row in open(file):
        if row.startswith('Score:'):
            row_list = row.split(' ')
            score = float(row_list[1][:-1])
            density = float(row_list[3][:-1])
        elif row.startswith('Configuration:'):
            is_conf = True
        elif is_conf:
            is_conf = False
            config = json.loads(row.replace('\'', '"'))
            # print(score, density, config)
            yield score, density, config

In [4]:
device = torch.device('cpu')

In [5]:
model = DetectMultiBackend('yolov5m.pt', device=device)
model.model.fuse().eval()
model = AutoShape(model)
model.ts = False

Fusing layers... 
[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [21]:
file = Path('./data/yolov5s.log')

In [22]:
# Small
target = {'model.model.model.8.cv3.conv': 8, 'model.model.model.8.cv2.conv': 14, 'model.model.model.6.cv2.conv': 3, 'model.model.model.13.cv3.conv': 5, 'model.model.model.23.m.0.cv1.conv': 6, 'model.model.model.10.conv': 7, 'model.model.model.21.conv': 12, 'model.model.model.7.conv': 14, 'model.model.model.13.cv2.conv': 1, 'model.model.model.18.conv': 5, 'model.model.model.6.m.1.cv1.conv': 2, 'model.model.model.14.conv': 1}

# Medium
# target = {'model.model.model.13.cv2.conv': 2, 'model.model.model.6.m.2.cv1.conv': 5, 'model.model.model.6.m.1.cv1.conv': 6, 'model.model.model.8.cv3.conv': 6, 'model.model.model.6.cv3.conv': 3, 'model.model.model.6.m.5.cv1.conv': 2, 'model.model.model.6.m.4.cv1.conv': 10, 'model.model.model.6.m.0.cv1.conv': 5, 'model.model.model.8.m.0.cv1.conv': 12, 'model.model.model.13.m.0.cv2.conv': 3, 'model.model.model.23.m.0.cv2.conv': 5, 'model.model.model.13.m.1.cv1.conv': 5, 'model.model.model.8.cv2.conv': 15, 'model.model.model.7.conv': 13, 'model.model.model.18.conv': 8, 'model.model.model.13.cv1.conv': 2, 'model.model.model.13.m.1.cv2.conv': 3, 'model.model.model.21.conv': 2, 'model.model.model.4.m.2.cv1.conv': 1, 'model.model.model.8.m.1.cv1.conv': 9, 'model.model.model.6.m.3.cv1.conv': 5, 'model.model.model.4.m.0.cv1.conv': 1, 'model.model.model.0.conv': 1, 'model.model.model.23.m.0.cv1.conv': 5, 'model.model.model.10.conv': 3, 'model.model.model.13.cv3.conv': 3, 'model.model.model.14.conv': 1, 'model.model.model.6.cv2.conv': 1, 'model.model.model.20.m.1.cv1.conv': 2}

In [36]:
configs = []
for i, (score, density, config) in enumerate(get_next(file)):
    copy_config = {}
    for key in config:
        copy_config[key.replace('model.model.', '')] = config[key]
    if i % 9 == 0 or i == 299:
        configs.append(copy_config)
print(len(configs))

35


In [37]:
configs

[{'model.8.cv3.conv': 1},
 {'model.8.cv3.conv': 2,
  'model.8.cv2.conv': 3,
  'model.6.cv2.conv': 1,
  'model.13.cv3.conv': 2,
  'model.23.m.0.cv1.conv': 2},
 {'model.8.cv3.conv': 3,
  'model.8.cv2.conv': 4,
  'model.6.cv2.conv': 2,
  'model.13.cv3.conv': 3,
  'model.23.m.0.cv1.conv': 2,
  'model.10.conv': 1,
  'model.21.conv': 2,
  'model.7.conv': 2},
 {'model.8.cv3.conv': 3,
  'model.8.cv2.conv': 5,
  'model.6.cv2.conv': 3,
  'model.13.cv3.conv': 3,
  'model.23.m.0.cv1.conv': 4,
  'model.10.conv': 1,
  'model.21.conv': 2,
  'model.7.conv': 5,
  'model.13.cv2.conv': 1,
  'model.18.conv': 1},
 {'model.8.cv3.conv': 3,
  'model.8.cv2.conv': 8,
  'model.6.cv2.conv': 3,
  'model.13.cv3.conv': 3,
  'model.23.m.0.cv1.conv': 5,
  'model.10.conv': 1,
  'model.21.conv': 4,
  'model.7.conv': 6,
  'model.13.cv2.conv': 1,
  'model.18.conv': 3},
 {'model.8.cv3.conv': 4,
  'model.8.cv2.conv': 10,
  'model.6.cv2.conv': 3,
  'model.13.cv3.conv': 3,
  'model.23.m.0.cv1.conv': 5,
  'model.10.conv': 2,
 

In [7]:
sum(config.values())

601

In [8]:
config.keys()

dict_keys(['model.model.model.13.cv2.conv', 'model.model.model.6.m.2.cv1.conv', 'model.model.model.6.m.1.cv1.conv', 'model.model.model.8.cv3.conv', 'model.model.model.6.cv3.conv', 'model.model.model.6.m.5.cv1.conv', 'model.model.model.6.m.4.cv1.conv', 'model.model.model.6.m.0.cv1.conv', 'model.model.model.8.m.0.cv1.conv', 'model.model.model.13.m.0.cv2.conv', 'model.model.model.23.m.0.cv2.conv', 'model.model.model.13.m.1.cv1.conv', 'model.model.model.8.cv2.conv', 'model.model.model.7.conv', 'model.model.model.18.conv', 'model.model.model.13.cv1.conv', 'model.model.model.13.m.1.cv2.conv', 'model.model.model.21.conv', 'model.model.model.4.m.2.cv1.conv', 'model.model.model.8.m.1.cv1.conv', 'model.model.model.6.m.3.cv1.conv', 'model.model.model.4.m.0.cv1.conv', 'model.model.model.0.conv', 'model.model.model.23.m.0.cv1.conv', 'model.model.model.10.conv', 'model.model.model.13.cv3.conv', 'model.model.model.14.conv', 'model.model.model.6.cv2.conv', 'model.model.model.20.m.1.cv1.conv', 'model.m

In [16]:
for key in config.keys():
    for name, conv in model.named_modules():
        if name == key:
            # print(conv.out_channels, conv.in_channels, config[key] * 5, 100 * (config[key] * 5) / conv.out_channels)
            print(torch.mean(torch.abs(conv.weight.data)), 100 * (config[key] * 5) / conv.out_channels)

tensor(0.05409) 15.625
tensor(0.10083) 49.479166666666664
tensor(0.11985) 52.083333333333336
tensor(0.02665) 16.276041666666668
tensor(0.02859) 14.322916666666666
tensor(0.04776) 39.0625
tensor(0.06574) 46.875
tensor(0.13281) 39.0625
tensor(0.10529) 39.0625
tensor(0.02338) 15.625
tensor(0.01042) 40.364583333333336
tensor(0.10485) 18.229166666666668
tensor(0.05089) 49.479166666666664
tensor(0.02060) 20.182291666666668
tensor(0.00633) 98.95833333333333
tensor(0.04868) 13.020833333333334
tensor(0.02440) 23.4375
tensor(0.00443) 44.270833333333336
tensor(0.13476) 26.041666666666668
tensor(0.06077) 31.25
tensor(0.07895) 31.25
tensor(0.25754) 46.875
tensor(1.17542) 10.416666666666666
tensor(0.04587) 57.291666666666664
tensor(0.04630) 28.645833333333332
tensor(0.07219) 20.833333333333332
tensor(0.04668) 23.4375
tensor(0.09325) 13.020833333333334
tensor(0.08802) 57.291666666666664
tensor(0.09775) 26.041666666666668
tensor(0.15449) 5.208333333333333
tensor(0.05309) 22.135416666666668
tensor(0.05

In [10]:
density

84.35

In [11]:
score

0.07289452105760574